In [ ]:
# 不使用
# 特徴量を結合
# Mora,MFeature_0,...,MFeature_767,Feature_0,...,Feature_767,Accent
# Mora,Accent列が無い
import pandas as pd
import os

# 特徴量とアクセント情報を結合して保存する関数
def merge_and_save_features(input_feature_dir, input_embedding_dir, output_dir, num_files=None):
    # 入力ディレクトリ内のすべてのファイルを取得
    feature_files = sorted([f for f in os.listdir(input_feature_dir) if f.endswith(".csv")])  # ファイル名でソート
    embedding_files = sorted([f for f in os.listdir(input_embedding_dir) if f.endswith(".csv")])  # ファイル名でソート
    
    if num_files is not None:
        feature_files = feature_files[:num_files]  # 最初のnum_files個のファイルだけを処理
        embedding_files = embedding_files[:num_files]

    for feature_file, embedding_file in zip(feature_files, embedding_files):
        # 各ファイルのパス
        feature_file_path = os.path.join(input_feature_dir, feature_file)
        embedding_file_path = os.path.join(input_embedding_dir, embedding_file)
        
        # CSVファイルを読み込む
        feature_df = pd.read_csv(feature_file_path)
        embedding_df = pd.read_csv(embedding_file_path)
        
        # basic5000_features_with_accent から全列を抽出
        feature_columns = [col for col in feature_df.columns if col != "Mora"]  # Mora以外の列（FeatureとAccentを含む）
        feature_columns += ["Mora", "Accent"]  # MoraとAccentを追加
        feature_df = feature_df[feature_columns]  # Mora, Feature_0, ..., Feature_768, Accent

        # processed_embeddings_karam から MFeature_0, ..., MFeature_768 を抽出
        embedding_columns = [col for col in embedding_df.columns if col.startswith("MFeature_")]  # MFeature_0 から MFeature_768
        embedding_df = embedding_df[embedding_columns]  # MFeature_0, MFeature_1, ..., MFeature_768
        
        # Mora列を削除せずに特徴量を結合
        merged_df = pd.concat([feature_df, embedding_df], axis=1)  # Mora列を保持して結合
        
        # 結果を保存する
        output_file_path = os.path.join(output_dir, feature_file)
        merged_df.to_csv(output_file_path, index=False)
        print(f"Saved: {output_file_path}")

# 実行部分
input_feature_directory = "./basic5000_features_with_accent"  # 特徴量が保存されているディレクトリ
input_embedding_directory = "./processed_embeddings_karam"  # 特徴量が保存されているディレクトリ、これ何？、MF,BERTで抽出した特徴量
output_directory = "./merged_features"  # 結合後の特徴量を保存するディレクトリ

# 出力ディレクトリが存在しない場合は作成
os.makedirs(output_directory, exist_ok=True)

num_files_to_process = 5  # 任意のファイル数（例: 最初の5ファイルを処理）

# 特徴量とアクセント情報を結合して保存
merge_and_save_features(input_feature_directory, input_embedding_directory, output_directory, num_files=num_files_to_process)

FileNotFoundError: [Errno 2] No such file or directory: './processed_embeddings_karam'

In [7]:
# step1;Mora only
import pandas as pd
import os

# Mora列だけを抽出して保存する関数
def extract_and_save_mora(input_feature_dir, output_dir, num_files=None):
    # 入力ディレクトリ内のすべてのファイルを取得
    feature_files = sorted([f for f in os.listdir(input_feature_dir) if f.endswith(".csv")])  # ファイル名でソート
    
    if num_files is not None:
        feature_files = feature_files[:num_files]  # 最初のnum_files個のファイルだけを処理

    for feature_file in feature_files:
        # 各ファイルのパス
        feature_file_path = os.path.join(input_feature_dir, feature_file)
        
        # CSVファイルを読み込む
        feature_df = pd.read_csv(feature_file_path)
        
        # Mora列だけを抽出
        mora_df = feature_df[["Mora"]]  # Mora列だけ
        
        # 結果を保存する
        output_file_path = os.path.join(output_dir, feature_file)
        mora_df.to_csv(output_file_path, index=False)
        print(f"Saved: {output_file_path}")

# 実行部分
input_feature_directory = "./basic5000_features_with_accent"  # 特徴量が保存されているディレクトリ
output_directory = "./mora_only"  # Mora列を保存するディレクトリ

# 出力ディレクトリが存在しない場合は作成
os.makedirs(output_directory, exist_ok=True)

num_files_to_process = 3  # 任意のファイル数（例: 最初の5ファイルを処理）

# Mora列を抽出して保存
extract_and_save_mora(input_feature_directory, output_directory, num_files=num_files_to_process)

Saved: ./mora_only/alignment_0001.csv
Saved: ./mora_only/alignment_0002.csv
Saved: ./mora_only/alignment_0003.csv


In [ ]:
# step2;Mora and MF
# MFがない！
import pandas as pd
import os

# MFeature_0 ～ MFeature_767 を mora_only に追加する関数
def add_mfeatures_to_mora(mora_dir, input_embedding_dir, num_files=None):
    # mora_onlyディレクトリ内のすべてのファイルを取得
    mora_files = sorted([f for f in os.listdir(mora_dir) if f.endswith(".csv")])  # ファイル名でソート
    
    if num_files is not None:
        mora_files = mora_files[:num_files]  # 最初のnum_files個のファイルだけを処理

    for mora_file in mora_files:
        # 各ファイルのパス
        mora_file_path = os.path.join(mora_dir, mora_file)
        embedding_file_name = f"embeddings_{mora_file}"  # processed_embeddings_karam フォルダ内の対応するファイル名
        embedding_file_path = os.path.join(input_embedding_dir, embedding_file_name)
        
        # Mora列とFeature_0 ～ Feature_767 を含んだファイルを読み込む
        mora_df = pd.read_csv(mora_file_path)
        
        # processed_embeddings_karam から MFeature_0 ～ MFeature_767 列を読み込む
        embedding_df = pd.read_csv(embedding_file_path)
        
        # MFeature_0 から MFeature_767 列が存在するかチェック
        embedding_columns = [f"MFeature_{i}" for i in range(0, 768)]  # MFeature_0 から MFeature_767 の列名
        available_embedding_columns = [col for col in embedding_columns if col in embedding_df.columns]  # 存在する列だけを選択
        
        # MFeature_0 ～ MFeature_767 の列を抽出（存在するものだけ）
        embedding_df_selected = embedding_df[available_embedding_columns]
        
        # MoraとMFeature_0 ～ MFeature_767 を結合
        result_df = pd.concat([mora_df, embedding_df_selected], axis=1)
        
        # 結果を同じmora_onlyディレクトリに保存する
        result_df.to_csv(mora_file_path, index=False)
        print(f"Saved: {mora_file_path}")

# 実行部分
mora_only_directory = "./mora_only"  # Mora列のみが保存されているディレクトリ
input_embedding_directory = "./processed_embeddings_karam"  # MFが保存されているディレクトリ

num_files_to_process = 3  # 任意のファイル数（例: 最初の5ファイルを処理）

# Mora列とMFeature_0 ～ MFeature_767を結合して保存
add_mfeatures_to_mora(mora_only_directory, input_embedding_directory, num_files=num_files_to_process)

FileNotFoundError: [Errno 2] No such file or directory: './processed_embeddings_karam/embeddings_alignment_0001.csv'

In [28]:
# step3;Mora and MF and F
import pandas as pd
import os

# Mora列にFeature_0 ～ Feature_767 を追加する関数
def add_features_to_mora(mora_dir, input_feature_dir, num_files=None):
    # mora_onlyディレクトリ内のすべてのファイルを取得
    mora_files = sorted([f for f in os.listdir(mora_dir) if f.endswith(".csv")])  # ファイル名でソート
    
    if num_files is not None:
        mora_files = mora_files[:num_files]  # 最初のnum_files個のファイルだけを処理

    for mora_file in mora_files:
        # 各ファイルのパス
        mora_file_path = os.path.join(mora_dir, mora_file)
        feature_file_path = os.path.join(input_feature_dir, mora_file)  # 同じ名前のファイルを basic5000_features_with_accent から読み込む
        
        # Mora列のみを含んだファイルを読み込む
        mora_df = pd.read_csv(mora_file_path)
        
        # basic5000_features_with_accent から Feature_0 ～ Feature_767 列を読み込む
        feature_df = pd.read_csv(feature_file_path)
        
        # Feature_0 から Feature_767 列が存在するかチェック
        feature_columns = [f"Feature_{i}" for i in range(0, 768)]  # Feature_0 から Feature_767 の列名
        available_feature_columns = [col for col in feature_columns if col in feature_df.columns]  # 存在する列だけを選択
        
        # Feature_0 ～ Feature_767 の列を抽出（存在するものだけ）
        feature_df_selected = feature_df[available_feature_columns]
        
        # MoraとFeature_0 ～ Feature_767 を結合
        result_df = pd.concat([mora_df, feature_df_selected], axis=1)
        
        # 結果を同じmora_onlyディレクトリに保存する
        result_df.to_csv(mora_file_path, index=False)
        print(f"Saved: {mora_file_path}")

# 実行部分
mora_only_directory = "./mora_only"  # Mora列が保存されているディレクトリ
input_feature_directory = "./basic5000_features_with_accent"  # 特徴量が保存されているディレクトリ

num_files_to_process = 5000  # 任意のファイル数（例: 最初の5ファイルを処理）

# Mora列とFeature_0 ～ Feature_767を結合して保存
add_features_to_mora(mora_only_directory, input_feature_directory, num_files=num_files_to_process)

Saved: ./mora_only/alignment_0001.csv
Saved: ./mora_only/alignment_0002.csv
Saved: ./mora_only/alignment_0003.csv
Saved: ./mora_only/alignment_0004.csv
Saved: ./mora_only/alignment_0005.csv
Saved: ./mora_only/alignment_0006.csv
Saved: ./mora_only/alignment_0007.csv
Saved: ./mora_only/alignment_0008.csv
Saved: ./mora_only/alignment_0009.csv
Saved: ./mora_only/alignment_0010.csv
Saved: ./mora_only/alignment_0011.csv
Saved: ./mora_only/alignment_0012.csv
Saved: ./mora_only/alignment_0013.csv
Saved: ./mora_only/alignment_0014.csv
Saved: ./mora_only/alignment_0015.csv
Saved: ./mora_only/alignment_0016.csv
Saved: ./mora_only/alignment_0017.csv
Saved: ./mora_only/alignment_0018.csv
Saved: ./mora_only/alignment_0019.csv
Saved: ./mora_only/alignment_0020.csv
Saved: ./mora_only/alignment_0021.csv
Saved: ./mora_only/alignment_0022.csv
Saved: ./mora_only/alignment_0023.csv
Saved: ./mora_only/alignment_0024.csv
Saved: ./mora_only/alignment_0025.csv
Saved: ./mora_only/alignment_0026.csv
Saved: ./mor

In [ ]:
# step4;Mora, MF, F and Accent
# これで良い！
import pandas as pd
import os

# Accent列をmora_onlyに追加する関数
def add_accent_to_mora(mora_dir, input_feature_dir, num_files=None):
    # mora_onlyディレクトリ内のすべてのファイルを取得
    mora_files = sorted([f for f in os.listdir(mora_dir) if f.endswith(".csv")])  # ファイル名でソート
    
    if num_files is not None:
        mora_files = mora_files[:num_files]  # 最初のnum_files個のファイルだけを処理

    for mora_file in mora_files:
        # 各ファイルのパス
        mora_file_path = os.path.join(mora_dir, mora_file)
        
        # basic5000_features_with_accent からのファイル名（alignment_0001.csv）を指定
        feature_file_path = os.path.join(input_feature_dir, "alignment_0001.csv")
        
        # Mora列とMFeature_0 ～ MFeature_767 を含んだファイルを読み込む
        mora_df = pd.read_csv(mora_file_path)
        
        # basic5000_features_with_accent から Feature_0 ～ Feature_767 列とAccent列を読み込む
        feature_df = pd.read_csv(feature_file_path)
        
        # Accent列が存在するか確認
        if 'Accent' not in feature_df.columns:
            print(f"Warning: 'Accent' column not found in {feature_file_path}. Available columns: {feature_df.columns}")
            continue
        
        # Feature_0 から Feature_767 列が存在するかチェック
        feature_columns = [f"Feature_{i}" for i in range(0, 768)]  # Feature_0 から Feature_767 の列名
        available_feature_columns = [col for col in feature_columns if col in feature_df.columns]  # 存在する列だけを選択
        
        # Feature_0 ～ Feature_767 の列を抽出（存在するものだけ）
        feature_df_selected = feature_df[available_feature_columns]
        
        # Accent列の抽出
        accent_column = feature_df['Accent']
        
        # MoraとFeature_0 ～ Feature_767、Accentを結合
        result_df = pd.concat([mora_df, feature_df_selected, accent_column], axis=1)
        
        # 結果を同じmora_onlyディレクトリに保存する
        result_df.to_csv(mora_file_path, index=False)
        print(f"Saved: {mora_file_path}")

# 実行部分
mora_only_directory = "./mora_only"  # Mora列が保存されているディレクトリ
input_feature_directory = "./basic5000_features_with_accent"  # basic5000の特徴量が保存されているディレクトリ

num_files_to_process = 5000  # 任意のファイル数（例: 最初の5ファイルを処理）

# Mora列とFeature_0 ～ Feature_767を結合してAccent列を追加
add_accent_to_mora(mora_only_directory, input_feature_directory, num_files=num_files_to_process)

Saved: ./mora_only/alignment_0001.csv
Saved: ./mora_only/alignment_0002.csv
Saved: ./mora_only/alignment_0003.csv
Saved: ./mora_only/alignment_0004.csv
Saved: ./mora_only/alignment_0005.csv
Saved: ./mora_only/alignment_0006.csv
Saved: ./mora_only/alignment_0007.csv
Saved: ./mora_only/alignment_0008.csv
Saved: ./mora_only/alignment_0009.csv
Saved: ./mora_only/alignment_0010.csv
Saved: ./mora_only/alignment_0011.csv
Saved: ./mora_only/alignment_0012.csv
Saved: ./mora_only/alignment_0013.csv
Saved: ./mora_only/alignment_0014.csv
Saved: ./mora_only/alignment_0015.csv
Saved: ./mora_only/alignment_0016.csv
Saved: ./mora_only/alignment_0017.csv
Saved: ./mora_only/alignment_0018.csv
Saved: ./mora_only/alignment_0019.csv
Saved: ./mora_only/alignment_0020.csv
Saved: ./mora_only/alignment_0021.csv
Saved: ./mora_only/alignment_0022.csv
Saved: ./mora_only/alignment_0023.csv
Saved: ./mora_only/alignment_0024.csv
Saved: ./mora_only/alignment_0025.csv
Saved: ./mora_only/alignment_0026.csv
Saved: ./mor

In [3]:
# 修正版
# MF only
import pandas as pd
import os

# processed_embeddings_karam から MFeature_0 ～ MFeature_767 を抽出し、結果を保存する関数
def extract_features_and_save(input_embedding_dir, output_dir, num_files=None):
    # processed_embeddings_karam 内のすべてのファイルを取得し、ソート
    embedding_files = sorted([f for f in os.listdir(input_embedding_dir) if f.endswith(".csv")])
    
    if num_files is not None:
        embedding_files = embedding_files[:num_files]  # 最初の num_files 個のファイルを処理

    # 出力ディレクトリが存在しない場合は作成
    os.makedirs(output_dir, exist_ok=True)

    for embedding_file in embedding_files:
        # 各ファイルのパス
        embedding_file_path = os.path.join(input_embedding_dir, embedding_file)
        
        # ファイルを読み込む
        embedding_df = pd.read_csv(embedding_file_path)
        
        # MFeature_0 ～ MFeature_767 の列を抽出
        feature_columns = [f"MFeature_{i}" for i in range(768)]
        available_feature_columns = [col for col in feature_columns if col in embedding_df.columns]  # 存在する列だけを選択
        
        # データフレームをフィルタリング
        result_df = embedding_df[available_feature_columns]
        
        # 保存先パスを生成
        output_file_path = os.path.join(output_dir, embedding_file)
        
        # 抽出した特徴量を保存
        result_df.to_csv(output_file_path, index=False)
        print(f"Saved: {output_file_path}")

# 実行部分
input_embedding_directory = "./processed_embeddings_karam"  # 特徴量が保存されているディレクトリ
output_directory = "./merged_results"  # 結果を保存するディレクトリ

num_files_to_process = 3  # 任意のファイル数（例: 最初の5000ファイルを処理）

# 特徴量を抽出して保存
extract_features_and_save(input_embedding_directory, output_directory, num_files=num_files_to_process)

Saved: ./merged_results/embeddings_alignment_0001.csv
Saved: ./merged_results/embeddings_alignment_0002.csv
Saved: ./merged_results/embeddings_alignment_0003.csv


In [5]:
# MF and F
import pandas as pd
import os

# Mora列にFeature_0 ～ Feature_767 を追加する関数
def add_features_to_mora(mora_dir, input_feature_dir, num_files=None):
    # mora_onlyディレクトリ内のすべてのファイルを取得
    mora_files = sorted([f for f in os.listdir(mora_dir) if f.endswith(".csv")])  # ファイル名でソート
    
    if num_files is not None:
        mora_files = mora_files[:num_files]  # 最初のnum_files個のファイルだけを処理

    for mora_file in mora_files:
        # 各ファイルのパス
        mora_file_path = os.path.join(mora_dir, mora_file)
        feature_file_path = os.path.join(input_feature_dir, mora_file)  # 同じ名前のファイルを basic5000_features_with_accent から読み込む
        
        # Mora列のみを含んだファイルを読み込む
        mora_df = pd.read_csv(mora_file_path)
        
        # basic5000_features_with_accent から Feature_0 ～ Feature_767 列を読み込む
        feature_df = pd.read_csv(feature_file_path)
        
        # Feature_0 から Feature_767 列が存在するかチェック
        feature_columns = [f"Feature_{i}" for i in range(0, 768)]  # Feature_0 から Feature_767 の列名
        available_feature_columns = [col for col in feature_columns if col in feature_df.columns]  # 存在する列だけを選択
        
        # Feature_0 ～ Feature_767 の列を抽出（存在するものだけ）
        feature_df_selected = feature_df[available_feature_columns]
        
        # MoraとFeature_0 ～ Feature_767 を結合
        result_df = pd.concat([mora_df, feature_df_selected], axis=1)
        
        # 結果を同じmora_onlyディレクトリに保存する
        result_df.to_csv(mora_file_path, index=False)
        print(f"Saved: {mora_file_path}")

# 実行部分
mora_only_directory = "./merged_results"  # MF only
input_feature_directory = "./processed_embeddings_karam"  # 結果が保存されるディレクトリ

num_files_to_process = 3  # 任意のファイル数（例: 最初の5ファイルを処理）

add_features_to_mora(mora_only_directory, input_feature_directory, num_files=num_files_to_process)

Saved: ./merged_results/embeddings_alignment_0001.csv
Saved: ./merged_results/embeddings_alignment_0002.csv
Saved: ./merged_results/embeddings_alignment_0003.csv


In [56]:
# デバッグ用
# accent列だけを別ディレクトリに保存
import pandas as pd
import os

def extract_accent_column(accent_features_dir, output_dir, num_files=None):
    # basic5000_features_with_accentディレクトリ内のすべてのCSVファイルを取得
    feature_files = sorted([f for f in os.listdir(accent_features_dir) if f.endswith(".csv")])  # ファイル名でソート

    # 処理するファイル数を制限
    if num_files is not None:
        feature_files = feature_files[:num_files]

    # 出力ディレクトリが存在しない場合は作成
    os.makedirs(output_dir, exist_ok=True)

    for feature_file in feature_files:
        # ファイルパスを定義
        feature_file_path = os.path.join(accent_features_dir, feature_file)
        output_file_path = os.path.join(output_dir, feature_file)

        # ファイルを読み込む
        feature_df = pd.read_csv(feature_file_path)

        # Accent列が存在するか確認
        if 'Accent' not in feature_df.columns:
            print(f"Warning: 'Accent' column not found in {feature_file_path}. Skipping.")
            continue

        # Accent列を抽出
        accent_column = feature_df[['Accent']]

        # 結果を出力ディレクトリに保存
        accent_column.to_csv(output_file_path, index=False)
        print(f"Saved: {output_file_path}")


# 実行部分
accent_features_directory = "./basic5000_features_with_accent"  # Accent列が入ったファイルのディレクトリ
output_directory = "./output_results"  # 保存先ディレクトリ

num_files_to_process = None  # 処理するファイル数（None の場合すべてを処理）

# 関数を実行
extract_accent_column(accent_features_directory, output_directory, num_files_to_process)

Saved: ./output_results/alignment_0001.csv
Saved: ./output_results/alignment_0002.csv
Saved: ./output_results/alignment_0003.csv
Saved: ./output_results/alignment_0004.csv
Saved: ./output_results/alignment_0005.csv
Saved: ./output_results/alignment_0006.csv
Saved: ./output_results/alignment_0007.csv
Saved: ./output_results/alignment_0008.csv
Saved: ./output_results/alignment_0009.csv
Saved: ./output_results/alignment_0010.csv
Saved: ./output_results/alignment_0011.csv
Saved: ./output_results/alignment_0012.csv
Saved: ./output_results/alignment_0013.csv
Saved: ./output_results/alignment_0014.csv
Saved: ./output_results/alignment_0015.csv
Saved: ./output_results/alignment_0016.csv
Saved: ./output_results/alignment_0017.csv
Saved: ./output_results/alignment_0018.csv
Saved: ./output_results/alignment_0019.csv
Saved: ./output_results/alignment_0020.csv
Saved: ./output_results/alignment_0021.csv
Saved: ./output_results/alignment_0022.csv
Saved: ./output_results/alignment_0023.csv
Saved: ./ou

In [62]:
# やっとできた！
# MF, F, Ac
import pandas as pd
import os

def add_accent_and_save_to_new_dir(merged_dir, accent_dir, output_dir, num_files=None):
    # 出力ディレクトリが存在しない場合は作成
    os.makedirs(output_dir, exist_ok=True)

    # merged_results ディレクトリ内のすべてのファイルを取得
    merged_files = sorted([f for f in os.listdir(merged_dir) if f.endswith(".csv")])  # ファイル名でソート

    # 処理するファイル数を制限
    if num_files is not None:
        merged_files = merged_files[:num_files]

    for merged_file in merged_files:
        # 各ファイルのパス
        merged_file_path = os.path.join(merged_dir, merged_file)
        accent_file_path = os.path.join(accent_dir, merged_file.replace("embeddings_", ""))  # ファイル名を調整
        output_file_path = os.path.join(output_dir, merged_file)  # 出力先のファイルパス

        # merged_resultsのCSVファイルを読み込む
        merged_df = pd.read_csv(merged_file_path)

        # output_results の Accent 列を含むファイルを読み込む
        if not os.path.exists(accent_file_path):
            print(f"Accent file not found for {merged_file}. Skipping.")
            continue
        
        accent_df = pd.read_csv(accent_file_path)

        # Accent列が存在するか確認
        if 'Accent' not in accent_df.columns:
            print(f"Warning: 'Accent' column not found in {accent_file_path}. Skipping.")
            continue

        # Accent列を結合
        merged_df['Accent'] = accent_df['Accent']

        # 結果を新しいディレクトリに保存
        merged_df.to_csv(output_file_path, index=False)
        print(f"Saved: {output_file_path}")

# 実行部分
merged_results_directory = "./merged_results"  # merged_results のディレクトリ
accent_results_directory = "./output_results"  # Accent列が保存されているディレクトリ
output_merged_results_directory = "./output_merged_results"  # 結果を保存するディレクトリ

num_files_to_process = None  # 処理するファイル数（None の場合すべてを処理）

# 関数を実行
add_accent_and_save_to_new_dir(merged_results_directory, accent_results_directory, output_merged_results_directory, num_files=num_files_to_process)

Saved: ./output_merged_results/embeddings_alignment_0001.csv
Saved: ./output_merged_results/embeddings_alignment_0002.csv
Saved: ./output_merged_results/embeddings_alignment_0003.csv
Saved: ./output_merged_results/embeddings_alignment_0004.csv
Saved: ./output_merged_results/embeddings_alignment_0005.csv
Saved: ./output_merged_results/embeddings_alignment_0006.csv
Saved: ./output_merged_results/embeddings_alignment_0007.csv
Saved: ./output_merged_results/embeddings_alignment_0008.csv
Saved: ./output_merged_results/embeddings_alignment_0009.csv
Saved: ./output_merged_results/embeddings_alignment_0010.csv
Saved: ./output_merged_results/embeddings_alignment_0011.csv
Saved: ./output_merged_results/embeddings_alignment_0012.csv
Saved: ./output_merged_results/embeddings_alignment_0013.csv
Saved: ./output_merged_results/embeddings_alignment_0014.csv
Saved: ./output_merged_results/embeddings_alignment_0015.csv
Saved: ./output_merged_results/embeddings_alignment_0016.csv
Saved: ./output_merged_r